# Subsetting for Dr. Elahi's Proteomics Pilot

In [1]:
# API Code to pull REDCap Report

#!/usr/bin/env python
import requests
data = {
    'token': '', # Write in token before running
    'content': 'report',
    'format': 'csv',
    'report_id': '3739',
    'csvDelimiter': '',
    'rawOrLabel': 'raw',
    'rawOrLabelHeaders': 'raw',
    'exportCheckboxLabel': 'false',
    'returnFormat': 'csv'
}
r = requests.post('https://redcap.rutgers.edu/api/',data=data)
print('HTTP Status: ' + str(r.status_code))
print(r.text) # This prints the HTML style export, comment this line to stop it from printing.

import pandas as pd
from io import StringIO
df = pd.read_csv(StringIO(r.text))

data1 = df
data1 = data1.drop('redcap_repeat_instrument', axis=1)
data1 = data1.groupby(['seqid', 'redcap_repeat_instance']).first().reset_index()
data1.to_csv("/Users/markgluck/Desktop/REDCap/Proteomics Pilot Participant Selection/data/pilot_proteomics_data.csv", index=False) # Change file path before running

data1

# You don't have to run this chunk to use this Notebook, the dataset is "pilot_proteomics_data.csv" in the /data folder


HTTP Status: 200


,seqid,redcap_repeat_instance,subjectid,age_1,gender,education,education_yrs,abca7_80_riskcalc,apoe_riskcalc
0,A001,1,None,NaN,NaN,NaN,NaN,NaN,NaN
1,A002,1,None,NaN,NaN,NaN,NaN,NaN,NaN
2,A003,1,None,NaN,NaN,NaN,NaN,NaN,NaN
3,A003,5,AA_4R003_L,75.0,1.0,1.0,14.0,NaN,NaN
4,A004,1,None,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1246,L152,1,AA_3R152_L,NaN,0.0,1.0,13.0,NaN,NaN
1247,L248,1,AA_4RL248,NaN,NaN,NaN,NaN,NaN,NaN
1248,L259,1,None,NaN,NaN,NaN,NaN,NaN,NaN
1249,L270,1,None,NaN,NaN,NaN,NaN,NaN,NaN


In [1]:
library(readr)

fullset <- read_csv("/Users/markgluck/Desktop/REDCap/Proteomics Pilot Participant Selection/data/pilot_proteomics_data.csv",
    col_types = cols(
        seqid = col_character(),
        redcap_repeat_instance = col_number(),
        age_1 = col_number(),
        gender = col_double(),
        education = col_number(),
        education_yrs = col_number(),
        abca7_80_riskcalc = col_number(),
        apoe_riskcalc = col_number()
    ),
    na = c("-9", "")
)

spec(fullset)
summary(fullset)


cols(
  seqid = col_character(),
  redcap_repeat_instance = col_number(),
  subjectid = col_character(),
  age_1 = col_number(),
  gender = col_double(),
  education = col_number(),
  education_yrs = col_number(),
  abca7_80_riskcalc = col_number(),
  apoe_riskcalc = col_number()
)

    seqid           redcap_repeat_instance  subjectid             age_1      
 Length:1251        Min.   :1.000          Length:1251        Min.   :24.00  
 Class :character   1st Qu.:1.000          Class :character   1st Qu.:64.00  
 Mode  :character   Median :1.000          Mode  :character   Median :69.00  
                    Mean   :1.556                             Mean   :69.17  
                    3rd Qu.:2.000                             3rd Qu.:74.00  
                    Max.   :9.000                             Max.   :92.00  
                                                              NA's   :390    
     gender         education     education_yrs   abca7_80_riskcalc
 Min.   :0.0000   Min.   :0.000   Min.   : 6.00   Min.   :0.0000   
 1st Qu.:0.0000   1st Qu.:0.000   1st Qu.:12.00   1st Qu.:0.0000   
 Median :0.0000   Median :1.000   Median :14.00   Median :0.0000   
 Mean   :0.1911   Mean   :1.127   Mean   :13.81   Mean   :0.1034   
 3rd Qu.:0.0000   3rd Qu.:2.000   3r

In [19]:
# Subsetting for the ABCA7 Positive & Negative dataset
library(dplyr)
library(skimr)
library(readxl)

# This is the shipment spreadsheet from Elizabeth
mt_sinai <- read_excel("/Users/markgluck/Desktop/REDCap/Proteomics Pilot Participant Selection/data/MtSinai_Shipment.xlsx")

# APOE subsets
apoenegative <- fullset %>%
  filter(apoe_riskcalc == "0")

apoepositive <- fullset %>%
  filter(apoe_riskcalc == "1")
# select(subjectid, apoe_riskcalc, age_1, gender, education_yrs)

# ABCA7-80 subsets
abca7_positive <- fullset %>%
  filter(abca7_80_riskcalc == 1)

abca7_negative <- fullset %>%
  filter(abca7_80_riskcalc == 0) %>%
  filter(age_1 <= 78)

# These are the ID's for ABCA7-80 Positive and APOE Positive in the Mt Sinai shipment spreadsheet
# apoe_supplements <- inner_join(mt_sinai, apoepositive, by = "subjectid")
abca7_positive_mt_sinai <- inner_join(mt_sinai, abca7_positive, by = "subjectid")

# This line joins all Positive IDs, this is the spreadsheet I'll be using
# abca7_apoe_positive_pilots <- full_join(apoe_supplements, abca7_positive_mt_sinai)

write.csv(abca7_apoe_positive_pilots, "data/abca7_all_positive_pilots_2april2024_mzs.csv", row.names = FALSE) # Writes CSV to /data folder

summary(abca7_positive_mt_sinai)

# All ABCA7 negative participants which are also in the shipment set up for Mt. Sinai already
abca7_all_negative <- inner_join(mt_sinai, abca7_negative, by = "subjectid")

write.csv(abca7_all_negative, "data/abca7_all_negative_pilots_21april2024_mzs.csv", row.names = FALSE) # Writes CSV to /data folder

summary(abca7_all_negative)


  subjectid              Date                            Box 1      
 Length:11          Min.   :2022-09-01 00:00:00.00   Min.   :1.000  
 Class :character   1st Qu.:2023-01-24 00:00:00.00   1st Qu.:2.000  
 Mode  :character   Median :2023-02-23 00:00:00.00   Median :2.000  
                    Mean   :2023-02-16 19:38:10.91   Mean   :2.091  
                    3rd Qu.:2023-04-15 12:00:00.00   3rd Qu.:2.500  
                    Max.   :2023-06-06 00:00:00.00   Max.   :3.000  
                                                                    
  Position #        Sample Type           Aliquots Volume (mL)       
 Length:11          Length:11          Min.   :2   Length:11         
 Class :character   Class :character   1st Qu.:2   Class :character  
 Mode  :character   Mode  :character   Median :2   Mode  :character  
                                       Mean   :2                     
                                       3rd Qu.:2                     
                            

  subjectid              Date                         Box 1      
 Length:30          Min.   :2022-10-18 00:00:00   Min.   :1.000  
 Class :character   1st Qu.:2023-01-17 00:00:00   1st Qu.:1.000  
 Mode  :character   Median :2023-04-18 00:00:00   Median :2.000  
                    Mean   :2023-03-12 11:12:00   Mean   :2.167  
                    3rd Qu.:2023-05-17 12:00:00   3rd Qu.:3.000  
                    Max.   :2023-06-13 00:00:00   Max.   :3.000  
                                                                 
  Position #        Sample Type           Aliquots Volume (mL)       
 Length:30          Length:30          Min.   :2   Length:30         
 Class :character   Class :character   1st Qu.:2   Class :character  
 Mode  :character   Mode  :character   Median :2   Mode  :character  
                                       Mean   :2                     
                                       3rd Qu.:2                     
                                       Max.   :2    

In [18]:
View(abca7_positive_mt_sinai)


subjectid,Date,Box 1,Position #,Sample Type,Aliquots,Volume (mL),seqid,redcap_repeat_instance,age_1,gender,education,education_yrs,abca7_80_riskcalc,apoe_riskcalc
<chr>,<dttm>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
AA_R472_L,2022-09-01,1,"9,10",EDTA plasma,2,250 uL,A472,3,NA,0,2,NA,1,1
COV133,2022-11-01,1,"59,60",EDTA plasma,2,250 uL,C133,1,NA,0,2,NA,1,0
AA_R304_L,2023-01-17,2,"3,4",EDTA plasma,2,250 uL,A304,3,NA,NA,1,NA,1,1
COV150,2023-01-31,2,"19,20",EDTA plasma,2,250 uL,C150,1,NA,1,0,NA,1,1
COV152,2023-02-07,2,"23,24",EDTA plasma,2,250 uL,C152,1,NA,0,3,NA,1,1
COV161,2023-02-23,2,"41,42",EDTA plasma,2,250 uL,C161,1,76,0,2,NA,1,0
COV162,2023-02-23,2,"43,44",EDTA plasma,2,250 uL,C162,1,69,1,1,13,1,1
COV172,2023-03-28,2,"63,64",EDTA plasma,2,250 uL,C172,1,NA,NA,NA,NA,1,1
COV187,2023-05-04,3,"5,6",EDTA plasma,2,250 uL,C187,1,73,1,0,12,1,0


# 2 April 2024

Dropped the subsets for ABCA7-80 and APOE, going to propensity match a sample using SPSS w/ Bern

In [30]:
# Revising subsetting process for Bern
library(dplyr)
library(skimr)
library(readxl)

# This is the shipment spreadsheet from Elizabeth
mt_sinai <- read_excel("/Users/markgluck/Desktop/REDCap/Proteomics Pilot Participant Selection/data/MtSinai_Shipment.xlsx")

mt_sinai_w_genetics <- left_join(mt_sinai, fullset, by = "subjectid", relationship = "many-to-many")

mt_sinai_w_genetics_no_na <- inner_join(mt_sinai, fullset, by = "subjectid", relationship = "many-to-many") %>%
    filter(abca7_80_riskcalc >= 0)


View(mt_sinai_w_genetics_no_na)

# write.csv(mt_sinai_w_genetics, "data/matching_spss/mt_sinai_genetics.csv", row.names = FALSE)

write.csv(mt_sinai_w_genetics_no_na, "data/matching_spss/mt_sinai_genetics_no_na.csv", row.names = FALSE)


subjectid,Date,Box 1,Position #,Sample Type,Aliquots,Volume (mL),seqid,redcap_repeat_instance,age_1,gender,education,education_yrs,abca7_80_riskcalc,apoe_riskcalc
<chr>,<dttm>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
COV113,2022-08-30,1,"3,4",EDTA plasma,2,250 uL,C113,1,NA,0,1,NA,0,0
COV116,2022-09-01,1,"7,8",EDTA plasma,2,250 uL,C116,1,NA,0,3,NA,0,0
AA_R472_L,2022-09-01,1,"9,10",EDTA plasma,2,250 uL,A472,3,NA,0,2,NA,1,1
COV118,2022-09-13,1,"15,16",EDTA plasma,2,250 uL,C118,1,NA,0,2,NA,0,0
COV121,2022-10-04,1,"29,30",EDTA plasma,2,250 uL,C121,1,NA,0,1,NA,0,0
COV124,2022-10-13,1,"37,38",EDTA plasma,2,250 uL,C124,1,NA,1,2,NA,0,1
COV127,2022-10-13,1,"43,44",EDTA plasma,2,250 uL,C127,1,NA,0,0,NA,0,0
COV128,2022-10-18,1,"45,46",EDTA plasma,2,250 uL,C128,1,NA,1,0,NA,0,1
COV129,2022-10-18,1,"47,48",EDTA plasma,2,250 uL,C129,1,65,0,1,14,0,1


# Update from 2April2024
Re-ran the split according to Bern: no APOE, propensity matched by ABCA7-80

Then split the file by ABCA7-80 positive and negative and sent to Elizabeth/Robby/Bern/Mark

See data/matching_spss/propensity_matched for the output files and Excel files.

In [ ]:
### Garbage Code! ###


# apoe_supplements_subset <- sample_n(apoe_supplements, size = 20) # Takes random sample of APOE positive participants already in shipment.
# abca7_negative_subset <- sample_n(abca7_negative, size = 20) # Only need to run this once, it takes a random sample each time
# abca7_negative_subset <- read_csv("data/abca7_negative_pilot_participants.csv") # Use this to avoid running the fullset chunk
# supplemental_abca7_negative <- sample_n(abca7_negative, size = 20)# Only need to run this once, it takes a random sample each time

# View(abca7_negative_subset)
# View(supplemental_abca7_negative)
# View(abca7_positive)
# View(abca7_negative)


# Checking for duplicates in the Negative supplemental list for Elizabeth
# abca7_neg_allpilot <- full_join(supplemental_abca7_negative, abca7_negative_subset)

# a <- duplicated(abca7_neg_allpilot)
# print(a)

# No duplicates!




# These write the pre-existing CSV files

# ABCA7+ and ABCA7- participants
# write.csv(abca7_positive, "data/abca7_positive_pilot_participants.csv", row.names = FALSE)
# write.csv(abca7_negative_subset, "data/abca7_negative_pilot_participants.csv", row.names = FALSE)

# Writing additional participants for ABCA7+ and ABCA7-
# write.csv(supplemental_abca7_negative, "data/abca7_negative_supplemental_participants.csv", row.names = FALSE)
# write.csv(apoe_supplements, "data/abca7_positive_supplemental_participants.csv", row.names = FALSE)
